# Домашнее задание 3

Это домашнее задание по материалам второго семинаров. Дедлайн по отправке - 23:55 24 февраля. 

Домашнее задание выполняется в этом же Jupyter Notebook'e и присылается мне на почту: __beznosikov.an@phystech.edu__.

Решение каждой задачи необходимо поместить после её условия.

Файл должен называться: Фамилия_Имя_Optimization_HW_3

При полном запуске Вашего решения (Kernel -> Restart & Run All) все ячейки должны выполняться без ошибок. 

## Задача 1

Вновь рассмотрим задачу минимизации эмпирического риска:
\begin{equation}
\min_{w \in \mathbb{R}^d} \frac{1}{n} \sum\limits_{i=1}^n l (g(w, x_i), y_i).
\end{equation}

В прошлом задании работа шла с линейной модель $g(w, x) = w^T x$ и квадратичную функцию потерь $l(z, y) = (z-y)^2$. 

__(а)__ В дополнение к квадратичной функции потерь реализуйте логистическую/сигмоидную: $l(z,y) = \ln (1 + \exp(-yz))$ (__Важно: $y$ должен принимать значения $-1$ или $1$__). Выпишите градиент. Является ли новая задача регресии выпуклой? Оцените $L$ для новой функции потерь. 

1) Gradient
$$\nabla_w Loss(w) = \nabla_w \frac1n \sum\limits_{i=1}^n \ln(1 + \exp(-w^Tx_i y_i)) = \frac1n \sum\limits_{i=1}^n \frac{-x_i y_i\exp(-w^Tx_i y_i)}{1 + \exp(-w^Tx_i y_i)}$$

2) $L = \| \frac14 X_n^TX_n \|$ as the reference from https://web.eecs.umich.edu/~fessler/course/598/l/n-03-gd.pdf page 53 

3) Convexity: yes, the logistic regression is a convex optimization problem, because Logloss is a convex function



In [13]:
from sklearn.datasets import load_svmlight_file
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import my_optimization as opt

__(б)__ Возьмите датасет _mushrooms_ из прошлого задания. Проделайте следующие шаги из прошлого задания, только с логистической функцией потерь:

1) Разделите данные на две части: обучающую и тестовую.

2) Для обучающей части $X_{train}$, $y_{train}$ оцените константу $L$ задачи обучения/оптимизации.

3) Используя градиентный спуск, обучите новую модель (без ограничений и регуляризаций). Постройте график: точность от номера итерации.

4) Если в пункте 3) пришлось столкнуться с проблемами или просто необходимо улучшить точность, то добавьте ограничения или $\ell_2$-регуляризацию, как в прошлом ДЗ.

5) Сравните с результатами квадратичной функции потерь из прошлого ДЗ.

In [15]:
#1
dataset = "mushrooms.txt" 
data = load_svmlight_file(dataset)
X, y = data[0].toarray(), data[1]
n, d = X.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=True)

#2
L = np.linalg.norm(1/(4*n) * X_train.T @ X_train)
print("L =", L)

L = 2.013288493062904


In [17]:
#3    
acc_n_iter_dependency (1, 200, 5, "basic", X_train, X_test, y_train, y_test, L, d)

ValueError: operands could not be broadcast together with shapes (682416,) (6093,112) (682416,) 

In [ ]:
#4

In [ ]:
#5
acc_hw_2 = 0.9960

## Задача 2

__(a)__ Реализуйте метод тяжелого шарика. 

In [ ]:
def Momentum_Grad_Descent(n_iter, X, lr, gamma, y, w_0, n):
    points = []
    w_old = w_0
    for i in range(n_iter):
        grad = Get_grad (X, y, w_old, n)
        if i < 2:
            w_new = w_old - lr*grad 
        else:
            w_new = w_old - lr*grad - gamma*(points[i] - points[i - 1])
        w_old = w_new 
        points.append(w_old)
    return points

def Momentum_Draw_Graph (n_iter, X, lr, gamma, y, w_0, n, w_true, criteria): 
    points = Momentum_Grad_Descent(n_iter, X, lr, gamma, y, w_0, n)
    diff_arr, crit_arr, x = [], [], []

    for i in range(n_iter):
        x.append(i + 1)
        diff_arr.append(points[i] - w_true)
        if criteria == "x_criteria":
            crit_arr.append(np.linalg.norm(diff_arr[i]))
        elif criteria == "f_criteria":
            crit_arr.append(np.abs(Logloss(X, y, points[i], n) - Logloss(X, y, w_true, n)))

    y = crit_arr
    if criteria == "x_criteria":
        plt.plot(x, y, label = "||w^k - w*||")
    elif criteria == "f_criteria":
        plt.plot(x, y, label = "|f(w^k) - f(w*)|")

    plt.xlabel('n_iter')
    plt.ylabel('criteria')
    plt.title('Standard axis convergence')
    plt.legend()
    plt.show()

__(б)__ Решите задачу логистической регрессии с помощью метода тяжелого шарика (не забудьте разделить выборку на две части: обучающую и тестовую). Зафиксируйте шаг $\frac{1}{L}$ и перебирайте разные значения моментума от -1 до 1. Постройте график сходимости метода от числа итераций (критерий сходимости подберите самостоятельно) для различных значений моментума. Всегда ли сходимость является монотонной?

In [ ]:
#ответ
w_true = Grad_Descent(10**4, X_train, 1/L, y_train, np.ones(d), np.shape((X_train)[0])[-1] )
for gamma in [-1, -0.8, -0.6, -0.4, -0.2, 0, 0.2, 0.4, 0.6, 0.8, 1]:
    Momentum_Draw_Graph (100, X, 1/L, y, np.ones(d), np.shape(X_train)[0])[-1], n, gamma, w_true, "f_criteria") 

__(в)__ Для лучшего значения моментума постройте график зависимости точности модели на тестовой выборке от времени работы метода. Добавьте на этот же график сходимость градиентного спуска с шагом $\frac{1}{L}$. Сделайте вывод.

In [ ]:
#ответ

__(г)__ Если в пунктах (б) и (в) столкнулись с проблемами, попробуйте $\ell_2$-регуляризовать задачу или рассмотреть ее на ограниченном множестве.

In [ ]:
#ответ

__(д)__ Реализуйте ускоренный метод Нестерова (в форме Нестерова, а не который доказывали на семинаре). 

In [ ]:
#ответ

__(е)__ Решите задачу логистической регресии с помощью метода Нестерова (не забудьте разделить выборку на две части: обучающую и тестовую). Зафиксируйте шаг $\frac{1}{L}$ и перебирайте разные значения моментума от -1 до 1. Проверьте также значения моментума равные $\frac{k}{k+3}$, $\frac{k}{k+2}$, $\frac{k}{k+1}$ ($k$ - номер итерации), а если решаете сильно выпуклую задачу, то и $\frac{\sqrt{L} - \sqrt{\mu}}{\sqrt{L} + \sqrt{\mu}}$. Постройте график сходимости метода от числа итераций (критерий сходимости подберите самостоятельно) для различных значений моментума. Всегда ли сходимость является монотонной?

In [ ]:
#ответ

__(ж)__ Для лучшего значения моментума постройте график зависимости точности модели на тестовой выборке от времени работы метода. Добавьте этот график к графикам для тяжелого шарика и градиентного спуска из пункта (г). Сделайте итоговый вывод.

In [ ]:
#ответ

__Бонусные пункт__

__(з)__ Сделаем подбор константы $L$ адаптивным. Как упоминалось на семинаре, можно измерять локальную $L$, используя:
$$
f(y) \leq f(x^k) + \langle \nabla f(x^k), y - x^k \rangle + \frac{L}{2}\|x^k - y\|_2^2
$$
В частности, может подойти процедура:

```python
def backtracking_L(f, grad, x, h, L0, rho):
    L = L0
    fx = f(x)
    gradx = grad(x)
    while True:
        y = x - 1 / L * h
        if f(y) <= fx - 1 / L gradx.dot(h) + 1 / (2 * L) h.dot(h):
            break
        else:
            L = L * rho
    return L
```

Каким стоит взять __h__? __rho__ должно быть больше или меньше 1? __L0__ надо брать заведомо большим или маленьким?

In [ ]:
#ответ

__(и)__ Поэксперементируйте с этой процедурой, встроенной в подбор $L$ для шага градиентного спуска. В качестве задачи продолжайте рассматривать логистическую регрессию из Задачи 1. Аналогично встройте процедуру подбора $L$ в метод тяжелого шарика и ускоренный метод Нестерова. Постройте график сходимости метода от числа итераций (критерий сходимости подберите самостоятельно). Отобразите на этом графике градиентный спуск, тяжелый шарик и метод Нестерова с адаптивным шагом и шагом $\frac{1}{L}$ (всего 6 линий на графике). Сделайте вывод.

In [ ]:
#ответ

__(к)__ Постройте аналогичный пункту (и) график точности модели от времени.

In [ ]:
#ответ

__(л)__ В [работе](https://arxiv.org/pdf/1204.3982.pdf) представлена техника рестартов для подавления немонотонной сходимости Алгоритма 2 (метод Нестерова). Попробуйте повторить эксперименты авторов на $\ell_2$-регуляризованной квадратичной или логистической регресии. Возьмите параметр регуляризации $\lambda = L / 100$.

In [ ]:
#ответ